<div align="center">
  <a href="https://www.davivienda.com/wps/portal/personas/nuevo">
    <img src="https://upload.wikimedia.org/wikipedia/en/thumb/b/b1/Davivienda_logo.svg/1200px-Davivienda_logo.svg.png" alt="Logo" width="300" height="100">
  </a>

  <h2 align="center">Prueba Técnica Davivienda ADNe - Especialista I</h2>
  <h2 align="center">Notebook de Implementación Predicción de Precios</h2>
  <h4 align="center">Luvan Tabares</h4>
    
***

  <p align="left">
    Se requiere desarrollar un modelo predictivo para estimar el valor total de avaluo de viviendas utilizando herramientas avanzadas de análisis de datos y machine learning. El objetivo es procesar las características de los inmuebles presentes en la base de datos para identificar patrones que permitan predecir el precio de propiedades que aún no han sido evaluadas.
  <p align="left">
    Este modelo será de gran utilidad en varios procesos bancarios, tales como:
  <p align="left">
    - Originación: Determinación del valor comercial del inmueble a financiar, crucial para la aprobación final de créditos.
  <p align="left">
    - Retanqueo: Actualización del valor comercial de garantías existentes, necesaria para la aprobación de nuevos cupos de crédito.
  <p align="left">
    - Monitoreo del Portafolio de Garantías: Valoración continua del portafolio de garantías, asegurando el cumplimiento normativo y facilitando el análisis de riesgos del colateral.
  <p align="left">
    - Normalización de Cartera: Evaluación de préstamos existentes y acuerdos de refinanciamiento, así como daciones en pago.
    
  </p>
</div>

## Tabla de contenidos

1. [Instalación de librerías y llamado de librerías](#first) <br />
2. [Exploración de datos](#second) <br />
3. [Preparación para el modelo](#third) <br />
4. [Resultados](#fourth) <br />

## 1. Librerias a instalar <a id="first"></a>

In [1]:
!pip install -r requirements.txt

## Cargar las librerías necesarias

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import re
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

from sklearn.metrics import mean_absolute_percentage_error, precision_score, f1_score
import numpy as np
import pandas as pd


## 2. Exploración de datos <a id="second"></a>

### Se inicia cargando el archivo de precios de vivienda

In [3]:
# Define la ruta al archivo CSV que contiene los datos de entrenamiento
ruta_csv = 'data_original/precios_vivienda/train_precios_vivienda (1).csv'
# Carga los datos del archivo CSV en un DataFrame de pandas, especificando la codificación UTF-8
df = pd.read_csv(ruta_csv, encoding='utf-8')

### Se observa el dataframe, este contiene 11571 filas y 222 columnas

In [4]:
df

,Unnamed: 0,id,fecha_aprobación,objeto,motivo,proposito,tipo_avaluo,tipo_credito,tipo_subsidio,departamento_inmueble,...,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud
0,4112,5896,43090.624747,Remate,Remates,GarantÃ­a Hipotecaria,Remates,Vivienda,NaN,VALLE DEL CAUCA,...,"81968750,00",0,0,0,0,"145318750,00",2522304,"576134,95",0.000000,0.000000
1,7401,10570,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,QUINDÃO,...,0,157,78500000,No,0,713986654,257.23250000000002,2775647.14,-75.661152,4.544027
2,10223,14600,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,ANTIOQUIA,...,0,0,0,Si,0,270500000,259.4264,1042684.94,-75.584116,6.277020
3,4170,5967,43091.676139,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,VIS,CUNDINAMARCA,...,0,0,0,No,0,"84840000,00",252245,"336339,67",0.000000,0.000000
4,11073,15814,NaN,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,VIS,META,...,69306400,0,0,0,0,96346400,259.72770000000003,370951.58,-73.712370,3.565757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11566,11964,17047,NaN,OriginaciÃ³n,Leasing Habitacional,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,"BOGOTÃ, D. C.",...,0,0,0,No,0,709028000,259.57420000000002,2731504.13,-74.047845,4.711578
11567,5191,7419,43157.000000,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,ANTIOQUIA,...,0,0,0,Si,2193300,158356260,254.18279999999999,623001.48,-74.610458,10.463742
11568,5390,7687,NaN,OriginaciÃ³n,Leasing Habitacional,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,NaN,"BOGOTÃ, D. C.",...,0,0,0,No,0,572610000,254.4109,2250729.04,-74.109652,4.674083
11569,860,1247,42999.429115,Originación,Crédito hipotecario de vivienda,Garantía Hipotecaria,Hipotecario,Vivienda,NaN,"BOGOTÁ, D. C.",...,"97997000,00",0,0,0,0,"183290000,00",2517095,"728214,55",0.000000,0.000000


### Para comenzar se deben eliminar los valores nulos

In [5]:
# Calcula el número de valores nulos en cada columna del DataFrame 'df'
null_counts = df.isnull().sum()
# Ordena las columnas por su cantidad de valores nulos de mayor a menor
sorted_null_counts = null_counts.sort_values(ascending=False)

In [6]:
sorted_null_counts

tipo_subsidio                 10219
fecha_aprobación               6774
descripcion_uso_inmueble        225
descripcion_tipo_inmueble       218
descripcion_clase_inmueble      218
                              ...  
area_libre                        0
valor_total_avaluo                0
valor_uvr                         0
valor_avaluo_en_uvr               0
Longitud                          0
Length: 222, dtype: int64

### Revisando la tabla anterior se observa que tipo_subsidio y fecha_aprobación tienen una gran cantidad de valores nulos, si se eliminara fecha_aprobación se perdería casi la mitad de los datos y con el otro se perdería casi e toda la data, adicionalmente estos son variables que no impactarían en la predicción de un inmueble, por lo que inicialmente se eliminarán estas columndas

In [7]:
# Eliminar las columnas 'tipo_subsidio' y 'fecha_aprobación'
df = df.drop(columns=['tipo_subsidio', 'fecha_aprobación'])

In [8]:
# Se vuelve a revisar la cantidad de valores nulos
# Calcula el número de valores nulos en cada columna del DataFrame 'df'
null_counts = df.isnull().sum()
# Ordena las columnas por su cantidad de valores nulos de mayor a menor
sorted_null_counts = null_counts.sort_values(ascending=False)

In [9]:
sorted_null_counts

descripcion_uso_inmueble                225
descripcion_tipo_inmueble               218
descripcion_clase_inmueble              218
numero_garaje_1                          26
observaciones_generales_construccion     15
                                       ... 
area_libre                                0
valor_total_avaluo                        0
valor_uvr                                 0
valor_avaluo_en_uvr                       0
Longitud                                  0
Length: 220, dtype: int64

### Ahora se observa que las columnas con valores nulos no superan los 225, pero se probará si es válida la eliminación de dichos valores con la cantidad de filas del dataframe, actualmente se tiene 11571, si después de la eliminación no se pierde la mitad de la data se podría trabajar con este nuevo dataframe

In [10]:
# Longitud del dataframe actual
len(df)

11571

In [11]:
# Elimina todas las filas del DataFrame 'df' que contienen al menos un valor nulo
df = df.dropna()
# Imprime la longitud actual del DataFrame después de eliminar las filas con valores nulos
print(f"La nueva longitud del DataFrame es: {len(df)}")

La nueva longitud del DataFrame es: 11292


### Afortunadamente la cantidad de data perdida es mínima, por lo que se continuará el desarrollo con este dataframe. Lo siguiente es revisar la variable objetivo valor_total_avaluo, ya se eliminaron los valores nulos, pero aquí valores de 0 no nos sirven, cada vivienda debe tener su propio valor para poder crear el modelo, nunca una vivienda va a tener un valor de 0,

In [12]:
# Cuenta el número de filas en el DataFrame 'df' donde la columna 'valor_total_avaluo' tiene un valor de 0
count_zeros_valor_total_avaluo = (df['valor_total_avaluo'] == 0).sum()
# Imprime el número de filas encontradas con valor 0 en la columna 'valor_total_avaluo'
print(f'Número de filas con valor 0 en "valor_total_avaluo": {count_zeros_valor_total_avaluo}')

Número de filas con valor 0 en "valor_total_avaluo": 0


### No se tiene valores 0 en la columna valor_total_avaluo, cuando se revisó el csv original se detectó eso, pero en la eliminación de valores nulos estos valores también fueron eliminados, por lo que ahora valor_total_avaluo siempre tiene un valor. Lo siguiente es asegurarnos que todos los números realmente se traten como números y no como strings

In [13]:
# Se va a detectar cuáles columnas son númericas y cuáles son categóricas
# Define una función para clasificar las columnas de un DataFrame como categóricas o numéricas
def identify_column_types(df):
    categoricos = []  # Inicializa una lista vacía para almacenar los nombres de las columnas categóricas
    numericos = []  # Inicializa una lista vacía para almacenar los nombres de las columnas numéricas
    
    for column in df.columns:  # Itera sobre cada columna del DataFrame
        # Intenta convertir los valores de la columna actual a numéricos, asignando NaN a los valores no convertibles
        converted = pd.to_numeric(df[column], errors='coerce')
        
        # Cuenta el número de valores NaN en la columna convertida (valores no numéricos)
        num_categoricals = converted.isna().sum()
        # Cuenta el número de valores no NaN en la columna convertida (valores numéricos)
        num_numerics = (~converted.isna()).sum()
               
        # Decide si la columna es categórica o numérica basándose en la cantidad de valores NaN y no NaN
        if num_categoricals > num_numerics:  # Si hay más valores categóricos que numéricos
            categoricos.append(column)  # Añade el nombre de la columna a la lista de categóricos
        else:  # Si hay más valores numéricos que categóricos
            numericos.append(column)  # Añade el nombre de la columna a la lista de numéricos
    
    return categoricos, numericos  # Devuelve las listas de columnas categóricas y numéricas

# Llama a la función para identificar y listar las columnas categóricas y numéricas del DataFrame 'df'
categoricos, numericos = identify_column_types(df)

### Revisando las columnas detectadas como numéricas se observan algunos errores, aunque no serían errores del procedimiento, si no de como llenan la información de dichas columnas, como descripcion_tipo_inmueble tiene valores numéricos y categoricos, prevalece la cantidad de numéricos por que en vez de poner un string que refiera a que no hay información están colocando 0

In [14]:
### Son 120 columnas numéricas ahora el problema es identificar esas columnas que realmente son categóricas
len(numericos)

120

### Se va a detectar de las numéricas mal clasificadas el porcentaje de categóricas detectadas

In [15]:
# Define una función para calcular el porcentaje de valores nulos en columnas numéricas y clasificarlas potencialmente como categóricas
def categorize_column_percentage(numericos, df):
    # Inicializa un diccionario para almacenar los nombres de las columnas y sus porcentajes de valores nulos
    column_percentages = {}

    # Calcula el porcentaje de valores nulos para cada columna numérica
    for column in numericos:
        # Intenta convertir la columna a numéricos, tratando los no convertibles como NaN
        extract_categorical = pd.to_numeric(df[column], errors='coerce')
        # Cuenta el número de valores NaN en la columna, que podrían indicar valores categóricos
        sub_categoricals = extract_categorical.isna().sum()
        # Obtiene el número total de filas en el DataFrame
        total_df = len(df)
        # Calcula el porcentaje de valores NaN en la columna
        percentage = (sub_categoricals / total_df) * 100
        # Almacena el porcentaje calculado en el diccionario
        column_percentages[column] = percentage
    
    # Convierte el diccionario a un DataFrame para una mejor visualización
    percentages_df = pd.DataFrame(list(column_percentages.items()), columns=['Column', 'Percentage'])

    # Ordena el DataFrame por porcentaje de forma descendente
    sorted_percentages_df = percentages_df.sort_values(by='Percentage', ascending=False)

    # Devuelve el DataFrame ordenado
    return sorted_percentages_df

### Observando la extracción de porcentajes como valor seguro podríamos tomar que menos del 5% en error es considerado normal, por lo que columnas con un porcentaje superior al 5% deberían ser revisadas, en esta categoría entra tipo_vigilancia, garaje_servidumbre_1, garaje_paralelo_1, etc. Con estas columnas se realizarán dos acercamientos, o serán eliminadas del df por que no contienen información tan relevanto o por que tienen una gran mezcla entre números y categorías, y la otra es intentar limpiar la columna convirtiendo textos a números relevantes.

In [16]:
percentages = categorize_column_percentage(numericos, df)
percentages.head(25)

,Column,Percentage
19,tipo_vigilancia,47.476089
49,garaje_servidumbre_1,44.801629
48,garaje_paralelo_1,44.792774
47,garaje_doble_1,44.721927
46,garaje_cubierto_1,44.491676
98,metodo_valuacion_8,32.571732
117,valor_avaluo_en_uvr,31.473610
45,matricula_garaje_1,27.293659
115,valor_total_avaluo,26.647184
99,concepto_del_metodo_8,24.849451


### Sacamos las columnas con un porcentaje de categoricos mayor al 5%

In [17]:
columns_above_5 = percentages[percentages['Percentage'] > 5]['Column'].tolist()

In [18]:
columns_above_5

['tipo_vigilancia',
 'garaje_servidumbre_1',
 'garaje_paralelo_1',
 'garaje_doble_1',
 'garaje_cubierto_1',
 'metodo_valuacion_8',
 'valor_avaluo_en_uvr',
 'matricula_garaje_1',
 'valor_total_avaluo',
 'concepto_del_metodo_8',
 'descripcion_uso_inmueble',
 'descripcion_tipo_inmueble',
 'area_valorada',
 'descripcion_clase_inmueble',
 'valor_area_privada',
 'area_privada',
 'tipo_deposito',
 'metodo_valuacion_6',
 'metodo_valuacion_3',
 'area_garaje',
 'matricula_inmobiliaria_deposito_1',
 'observaciones_indice_ocupacion',
 'concepto_del_metodo_3',
 'garaje_servidumbre_2',
 'garaje_cubierto_2',
 'garaje_paralelo_2',
 'garaje_doble_2',
 'valor_area_construccion',
 'area_construccion',
 'concepto_del_metodo_6']

In [19]:
# Función para extraer valores únicos de las columnas especificadas
def extract_unique_values_from_list(df, columns_list):
    unique_values = {}
    for column in columns_list:
        if column in df.columns:
            # Convertir todos los valores a cadenas
            values = df[column].astype(str)
            # Extraer valores únicos y eliminar posibles valores no válidos
            unique_values[column] = sorted(set(values) - {'nan', 'None'})
        else:
            unique_values[column] = 'Columna no encontrada en el DataFrame'
    return unique_values

In [20]:
# Llamar a la función con la lista de columnas deseada
unique_values = extract_unique_values_from_list(df, columns_above_5)

In [21]:
# Función para contar los valores únicos por clave
def count_unique_values(unique_values_dict):
    counts = {}
    for key, values in unique_values_dict.items():
        counts[key] = len(values)
    return counts

# Función para ordenar los conteos de mayor a menor
def sort_by_count(counts_dict):
    return dict(sorted(counts_dict.items(), key=lambda item: item[1], reverse=True))

In [22]:
# Llamar a la función
unique_counts = count_unique_values(unique_values)
# Ordenar de mayor a menor
sorted_unique_counts = sort_by_count(unique_counts)

In [23]:
sorted_unique_counts

{'valor_avaluo_en_uvr': 11177,
 'valor_total_avaluo': 10823,
 'valor_area_privada': 7695,
 'area_valorada': 6884,
 'area_privada': 5498,
 'matricula_garaje_1': 3106,
 'valor_area_construccion': 2927,
 'area_construccion': 1984,
 'area_garaje': 1507,
 'descripcion_tipo_inmueble': 1364,
 'descripcion_clase_inmueble': 1363,
 'descripcion_uso_inmueble': 1020,
 'matricula_inmobiliaria_deposito_1': 872,
 'concepto_del_metodo_8': 701,
 'concepto_del_metodo_3': 206,
 'concepto_del_metodo_6': 145,
 'observaciones_indice_ocupacion': 138,
 'garaje_cubierto_1': 37,
 'metodo_valuacion_6': 27,
 'garaje_doble_1': 16,
 'garaje_cubierto_2': 11,
 'garaje_paralelo_1': 8,
 'tipo_vigilancia': 5,
 'metodo_valuacion_8': 5,
 'garaje_servidumbre_1': 4,
 'metodo_valuacion_3': 4,
 'garaje_doble_2': 4,
 'tipo_deposito': 3,
 'garaje_servidumbre_2': 3,
 'garaje_paralelo_2': 3}

## Correcciones necesarias
## Ajustes de valores numéricos
### Las siguientes columnas contienen valores que requieren la corrección de la coma decimal por un punto para su correcta conversión a float: valor_avaluo_en_uvr, valor_total_avaluo,valor_area_privada, area_valorada, valor_area_construccion, area_construccion, area_garaje, observaciones_indice_ocupacion. Además, es necesario eliminar los valores que sean 0 o excesivamente pequeños, así como aquellos que sean extremadamente grandes, para asegurar la validez de los datos.

## Limpieza de datos categóricos
### Las columnas garaje_cubierto_1, garaje_doble_1, garaje_cubierto_2, garaje_paralelo_1, garaje_doble_2, tipo_deposito, garaje_servidumbre_2 y garaje_paralelo_2 requieren limpieza, ya que contienen valores que indican la existencia de un garaje mediante la matrícula, o utilizan textos como "sí" y "no". Estos valores deben ser convertidos a un formato binario, donde 0 indica la ausencia y 1 la presencia del garaje.

### Normalización de tipos de vigilancia
## La columna tipo_vigilancia necesita limpieza y normalización para asegurar que los valores sean 0, 12 o 24, ya que actualmente se están utilizando textos en lugar de estos valores numéricos.

## Eliminación de columnas
## Columna area_privada
### La columna area_privada contiene un error, ya que mezcla el área del inmueble con características adicionales como sistemas contra incendios, jacuzzi, canchas, entre otros. Por esta razón, se procederá a eliminar esta columna.

## Columnas de matrícula
### Las columnas matricula_garaje_1 y matricula_inmobiliaria_deposito_1 solo contienen códigos de identificación catastral del inmueble y, por tanto, serán eliminadas.

## Descripción del tipo de inmueble
### La columna descripcion_tipo_inmueble presenta muchos valores de 0 para indicar la falta de descripción. Además, la variedad de descripciones diferentes es tan elevada que no se puede considerar como un dato consistente. Por lo tanto, esta columna será eliminada.

## Otras descripciones y métodos de valoración
### Las columnas descripcion_clase_inmueble, descripcion_uso_inmueble, metodo_valuacion_6, concepto_del_metodo_8, concepto_del_metodo_3, concepto_del_metodo_6, metodo_valuacion_8 y metodo_valuacion_3 presentan un problema similar al de descripcion_tipo_inmueble, con numerosos valores de 0 para indicar la ausencia de descripción. Por esta razón, estas columnas también serán eliminadas.

In [24]:
# Define una lista de nombres de columnas que se consideran irrelevantes o redundantes para el análisis o modelo de predicción
# Estas columnas serán eliminadas del DataFrame para simplificar el dataset y potencialmente mejorar el rendimiento del modelo
columnas_eliminar_1 = ["area_privada", 
                     "matricula_garaje_1" , 
                     "matricula_inmobiliaria_deposito_1", 
                     "descripcion_tipo_inmueble", 
                     "descripcion_clase_inmueble", 
                     "descripcion_uso_inmueble", 
                     "metodo_valuacion_6",
                     "concepto_del_metodo_8",
                     "concepto_del_metodo_3",
                     "concepto_del_metodo_6",
                     "metodo_valuacion_8",
                     "metodo_valuacion_3"]

### Eliminando columnas

In [25]:
df = df.drop(columns=columnas_eliminar_1)

In [26]:
df

,Unnamed: 0,id,objeto,motivo,proposito,tipo_avaluo,tipo_credito,departamento_inmueble,municipio_inmueble,barrio,...,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud
0,4112,5896,Remate,Remates,GarantÃ­a Hipotecaria,Remates,Vivienda,VALLE DEL CAUCA,TULUA,VICTORIA,...,"81968750,00",0,0,0,0,"145318750,00",2522304,"576134,95",0.000000,0.000000
1,7401,10570,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,QUINDÃO,ARMENIA,SECTOR CLINICAS,...,0,157,78500000,No,0,713986654,257.23250000000002,2775647.14,-75.661152,4.544027
2,10223,14600,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,ANTIOQUIA,MEDELLIN,ROBLEDO PILARICA,...,0,0,0,Si,0,270500000,259.4264,1042684.94,-75.584116,6.277020
3,4170,5967,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,CUNDINAMARCA,SOACHA,CIUDAD VERDE,...,0,0,0,No,0,"84840000,00",252245,"336339,67",0.000000,0.000000
4,11073,15814,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,META,GRANADA,BULEVAR ETAPA II,...,69306400,0,0,0,0,96346400,259.72770000000003,370951.58,-73.712370,3.565757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11566,11964,17047,OriginaciÃ³n,Leasing Habitacional,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,"BOGOTÃ, D. C.","BOGOTÃ, D. C.",Country Club,...,0,0,0,No,0,709028000,259.57420000000002,2731504.13,-74.047845,4.711578
11567,5191,7419,OriginaciÃ³n,CrÃ©dito hipotecario de vivienda,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,ANTIOQUIA,MEDELLIN,BOYACA LAS BRISAS,...,0,0,0,Si,2193300,158356260,254.18279999999999,623001.48,-74.610458,10.463742
11568,5390,7687,OriginaciÃ³n,Leasing Habitacional,GarantÃ­a Hipotecaria,Hipotecario,Vivienda,"BOGOTÃ, D. C.","BOGOTÃ, D. C.",Normandia,...,0,0,0,No,0,572610000,254.4109,2250729.04,-74.109652,4.674083
11569,860,1247,Originación,Crédito hipotecario de vivienda,Garantía Hipotecaria,Hipotecario,Vivienda,"BOGOTÁ, D. C.","BOGOTÁ, D. C.",Los Molinos - Rafeal U. U.,...,"97997000,00",0,0,0,0,"183290000,00",2517095,"728214,55",0.000000,0.000000


### Revisando de nuevo el dataframe podemos seleccionar otras columnas rápidamente que a consideración no son importantes como: Unnamed: 0, objeto, motivo, proposito, tipo_avaluo, tipo_credito.

In [27]:
# Define una lista de nombres de columnas a eliminar del DataFrame 'df'
columnas_eliminar_2 = ["Unnamed: 0", "objeto", "motivo", "proposito", "tipo_avaluo", "tipo_credito"]
# Elimina las columnas especificadas de 'df' y actualiza el DataFrame
df = df.drop(columns=columnas_eliminar_2)

### Primeramente se arreglará la columna más importante: valor_total_avaluo, que será nuestra variable objetivo

In [28]:
def safe_convert(value):
    # Try to convert the value to float
    try:
        # Replace commas with dots for decimal conversion and convert to float
        return float(str(value).replace(',', '.'))
    except Exception:
        # If conversion fails, return None
        return None


def convert_to_float(df, columns):
    # Iterate over the specified columns
    for column in columns:
        # Apply the safe_convert function to each element in the column
        df[column] = df[column].apply(safe_convert)
        # Use pd.to_numeric to ensure the column is numeric and handle errors by coercing non-numeric values to NaN
        df[column] = pd.to_numeric(df[column], errors='coerce')
        

In [29]:
# Se convierte a float la columna 'valor_total_avaluo'
convert_to_float(df, ['valor_total_avaluo'])

In [30]:
# Comprobar si todos los valores en la columna son numéricos
def is_numeric_column(series):
    try:
        series.astype(float)
        return True
    except ValueError:
        return False

# Verificar si la columna 'valor_total_avaluo' es numérica
is_numeric = is_numeric_column(df['valor_total_avaluo'])

if is_numeric:
    print("La columna 'valor_total_avaluo' es totalmente numérica.")
else:
    print("La columna 'valor_total_avaluo' contiene valores no numéricos.")

La columna 'valor_total_avaluo' es totalmente numérica.


In [31]:
# Inspeccionar los valores mínimos y máximos
print(df['valor_total_avaluo'].describe())
print("\n")

# Ver los valores únicos más bajos y más altos
print(df['valor_total_avaluo'].nsmallest(10))
print("\n")
print(df['valor_total_avaluo'].nlargest(10))

count    1.129200e+04
mean     2.252684e+09
std      7.744162e+10
min      0.000000e+00
25%      1.126758e+08
50%      1.749518e+08
75%      2.893915e+08
max      5.744320e+12
Name: valor_total_avaluo, dtype: float64


279     0.0
360     0.0
470     0.0
541     0.0
694     0.0
742     0.0
794     0.0
1030    0.0
1111    0.0
1117    0.0
Name: valor_total_avaluo, dtype: float64


8302     5.744320e+12
1715     4.750622e+12
9098     1.828220e+12
10499    1.648560e+12
1783     1.295616e+12
6702     1.035721e+12
7779     9.991965e+11
8572     9.084200e+11
1479     6.250000e+11
8374     4.887500e+11
Name: valor_total_avaluo, dtype: float64


In [32]:
df = df[df['valor_total_avaluo'] > 0]

In [33]:
# Calcular percentiles
percentile_1 = df['valor_total_avaluo'].quantile(0.01)
percentile_99 = df['valor_total_avaluo'].quantile(0.99)

### Usando los percentiles se nos sugiere usar un rango entre 46 millones de pesos y 1300 millones de pesos, aunque para el límite inferior lo veo
### acertado para el límite superior no lo veo tan acertado, ya que hay valores que superan los 1300 millones de pesos.
### Una forma de abordar esto es buscar las casas que superen los 1300 millones de pesos y ver el estrato, si es un estrado que no sea 6 o superior
### Muy probablemente fué un error de digitación y se puede eliminar.

limite_inferior = percentile_1
limite_superior = percentile_99* 0.6 # Se reduce el límite superior a 80% del percentil 99

print(f"Usando los percentiles 1 y 99, los límites inferior y superior son: {limite_inferior} y {limite_superior}")

# Para continuar se limpiará la columna estrato, una vez se encuentre limpia se podrá realizar el filtro

convert_to_float(df, ['estrato'])

Usando los percentiles 1 y 99, los límites inferior y superior son: 46434500.0 y 806113440.0


In [34]:
# Se revisa si la limpieza de estrato generó valores nulos:
null_counts = df.isnull().sum()
sorted_null_counts = null_counts.sort_values(ascending=False)
print(sorted_null_counts)
# Aquí estrato generó 5 valores nulos, está bien, por lo que serán eliminados los valores nulos
df = df.dropna()
# Con esto ya no hay valores nulos en el DataFrame
print(f"El dataframe cuenta con {len(df)} filas")
# Se siguen teniendo una buena cantidad de datos

estrato                5
id                     0
municipio_inmueble     0
barrio                 0
sector                 0
                      ..
valor_total_avaluo     0
valor_uvr              0
valor_avaluo_en_uvr    0
Longitud               0
Latitud                0
Length: 202, dtype: int64
El dataframe cuenta con 11146 filas


In [35]:
# Ahora con esto se aplica el filtro de los valores que superen los 1300 millones de pesos y que no sean de estrato 6 o superior, adicionalmente
# se filtrarán los valores que sean menores a 46 millones de pesos
print(f"DataFrame original tenía {len(df)} filas")
# Aplicar el filtro
df = df[df['valor_total_avaluo'] >= limite_inferior]

sub_df_limite_superior = df[df['valor_total_avaluo'] > limite_superior]
sub_df_limite_superior = sub_df_limite_superior[sub_df_limite_superior['estrato'] <= 4]
ids_raras = sub_df_limite_superior['id'].to_list()
df = df[~df['id'].isin(ids_raras)]

print(f"DataFrame después de aplicar los filtros tiene {len(df)} filas")

DataFrame original tenía 11146 filas
DataFrame después de aplicar los filtros tiene 10971 filas


## Re check para ver si quedó bien, poniendo el valor de casa de 1000 millones se observa que para la columna extrato solo tenemos como valores 5 y 6, cosa que se esperaría del comportamiento de la vivienda a día de hoy

In [36]:
# Filter the DataFrame 'df' to include only rows where 'valor_total_avaluo' is greater than 1 billion
re_filter = df[df['valor_total_avaluo']>1000000000]
# Count the occurrences of each unique value in the 'estrato' column of the filtered DataFrame
# This gives us an idea of the distribution of property strata in high-value properties
re_filter['estrato'].value_counts()

estrato
6.0    144
5.0     29
Name: count, dtype: int64

### Revisamos métricas de los precios, se observa que la desviación es exageradamente elevada, 71mil millones, esto debido a que el valor máximo es 5.7 billones?, este valor debe ser sacado por que es un valor demasiado atípico.

In [37]:
# Calculate basic statistics (count, mean, std, min, 25%, 50%, 75%, max) for the 'valor_total_avaluo' column
statistics = df['valor_total_avaluo'].describe()
# Format each statistic value to include commas as thousands separators and round to two decimal places
formatted_statistics = statistics.apply(lambda x: f"{x:,.2f}")
# Print the formatted statistics to the console
print(formatted_statistics)

count               10,971.00
mean         1,326,692,141.72
std         71,873,634,130.12
min             46,459,000.00
25%            116,052,760.00
50%            177,234,000.00
75%            290,403,000.00
max      5,744,320,000,008.00
Name: valor_total_avaluo, dtype: object


In [38]:
# Volvemos a calcular los percentiles
percentile_1 = df['valor_total_avaluo'].quantile(0.01)
percentile_99 = df['valor_total_avaluo'].quantile(0.99)

print(f"Usando los percentiles 1 y 99, los límites inferior y superior son: {percentile_1} y {percentile_99}")


Usando los percentiles 1 y 99, los límites inferior y superior son: 53551089.0 y 1217928739.9999993


### Se observa que la cantidad de valores por encima del límite superior que es 1.217.928.740 es de solo 110, aproximadamente el 1% de la data total de la muestra que ya ha sido filtrada, esto quiere decir que este modelo no podrá inferir en viviendas con valores superiores a este límite, si esto se desea se deben encontrar mucha más información de vivienda por encima de estos valores para poder crear un modelo que tenga en cuenta estos valores.

In [39]:
num_values_above_percentile_99 = df[df['valor_total_avaluo'] > percentile_99].shape[0]
print(f"El número de valores por encima del percentil 99 es: {num_values_above_percentile_99} que equivale al {num_values_above_percentile_99/len(df)*100:.2f}%")

El número de valores por encima del percentil 99 es: 110 que equivale al 1.00%


In [40]:
df = df[df['valor_total_avaluo'] <= percentile_99]
print(f"Después de aplicar el filtro, el DataFrame tiene {len(df)} filas")

Después de aplicar el filtro, el DataFrame tiene 10861 filas


In [41]:
# Calculate basic statistics (count, mean, std, min, 25%, 50%, 75%, max) for the 'valor_total_avaluo' column
statistics = df['valor_total_avaluo'].describe()
# Format each statistic value to include commas as thousands separators and round to two decimal places
formatted_statistics = statistics.apply(lambda x: f"{x:,.2f}")
# Print the formatted statistics to the console
print(formatted_statistics)

count           10,861.00
mean       231,994,064.31
std        176,436,570.74
min         46,459,000.00
25%        115,668,000.00
50%        175,760,000.00
75%        284,777,500.00
max      1,217,598,200.00
Name: valor_total_avaluo, dtype: object


### Ahora la desviación estándar es más lógica, con un promedio situado en 230 millones, que es realmente el valor de una casa ni muy cara ni muy barata

### Se vuelve a revisar la data

In [42]:
# Filtra las columnas numéricas para excluir aquellas en 'columnas_eliminar_1'
numericos_filtrados = [col for col in numericos if col not in columnas_eliminar_1]
# Vuelve a filtrar el resultado para excluir cualquier columna adicional en 'columnas_eliminar_2'
numericos_filtrados = [col for col in numericos_filtrados if col not in columnas_eliminar_2]

In [43]:
# Categoriza los valores en 'numericos_filtrados' basándose en su porcentaje en el DataFrame 'df'
percentages = categorize_column_percentage(numericos_filtrados, df)
# Muestra las primeras 25 filas del DataFrame 'percentages' para revisar los resultados de la categorización
percentages.head(25)

,Column,Percentage
15,tipo_vigilancia,47.472608
42,garaje_doble_1,45.133966
41,garaje_cubierto_1,45.133966
44,garaje_servidumbre_1,45.133966
43,garaje_paralelo_1,45.133966
104,valor_avaluo_en_uvr,31.424362
12,area_valorada,24.307154
90,valor_area_privada,19.970537
69,tipo_deposito,13.635945
91,area_garaje,8.203664


### Limpieza area_garaje, en este caso si puede ser 0 ya que indicaría que no hay garaje, y como el área de la casa a mayor sea mayor valor debería tener la casa

In [44]:
# Esta si no tiene problemas y será limpiada por si acaso, similar a 'valor_total_avaluo'
# Se convierte a float la columna 'valor_avaluo_en_uvr'
convert_to_float(df, ['area_garaje'])

### Limpieza de tipo_vigilancia

In [45]:
# Define una función para extraer el primer número encontrado en una cadena de texto
def limpiar_tipo_vigilancia(cadena):
    # Utiliza una expresión regular para buscar la primera secuencia de dígitos en la cadena
    match = re.search(r'\d+', cadena)
    # Si se encuentra una coincidencia, convierte la secuencia de dígitos a un entero y lo retorna
    # Si no se encuentra ninguna coincidencia, retorna None
    return int(match.group()) if match else None

# Limpiar la columna 'tipo_vigilancia' y extraer el número
df['tipo_vigilancia'] = df['tipo_vigilancia'].apply(limpiar_tipo_vigilancia)

### Limpieza garaje_doble_1, garaje_cubierto_1, garaje_servidumbre_1, garaje_paralelo_1, garaje_doble_2, garaje_paralelo_2, garaje_servidumbre_2, garaje_cubierto_2

In [46]:
# Define una función para limpiar los valores de la columna 'garaje_doble_1'
def limpiar_garaje(value):
    # Convierte el valor de entrada a una cadena para asegurar un procesamiento consistente
    value = str(value)
    # Si el valor es 'Si', retorna 1 (indicando la presencia de un garaje doble)
    if value == 'Si':
        return 1
    # Si el valor es 'No' o '0', retorna 0 (indicando la ausencia de un garaje doble)
    elif value == 'No' or value == '0':
        return 0
    # Para cualquier otro valor, retorna None para indicar un valor indefinido o faltante
    else:
        return None

# Aplica la función de limpieza a cada valor en la columna 'garaje_doble_1' del DataFrame 'df'
# Esto estandariza los valores de la columna a 1, 0, o None
df['garaje_doble_1'] = df['garaje_doble_1'].apply(limpiar_garaje)
df['garaje_cubierto_1'] = df['garaje_cubierto_1'].apply(limpiar_garaje)
df['garaje_servidumbre_1'] = df['garaje_servidumbre_1'].apply(limpiar_garaje)
df['garaje_paralelo_1'] = df['garaje_paralelo_1'].apply(limpiar_garaje)
df['garaje_doble_2'] = df['garaje_doble_2'].apply(limpiar_garaje)
df['garaje_paralelo_2'] = df['garaje_paralelo_2'].apply(limpiar_garaje)
df['garaje_servidumbre_2'] = df['garaje_servidumbre_2'].apply(limpiar_garaje)
df['garaje_cubierto_2'] = df['garaje_cubierto_2'].apply(limpiar_garaje)

### Limpieza valor_avaluo_en_uvr, valor_area_privada

In [47]:
# Contar cuántos valores son 0 en la columna 'valor_area_privada'
conteo = df['valor_area_privada'].apply(lambda x: x == 0 or x == '0').sum()

print(f"El número de valores 0 en 'valor_area_privada' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")

El número de valores 0 en 'valor_area_privada' es: 3143 que representa el 28.94% de los datos


In [48]:
# Esto es muy elevado, casi el 30% de la data tiene valores 0 en 'valor_area_privada', por lo que se eliminarán estos valores
df = df.drop(columns=['valor_area_privada'])

### Se hará el mismo cálculo para valor_avaluo_en_uvr

In [49]:
# Contar cuántos valores son 0 en la columna 'valor_area_privada'
conteo = df['valor_avaluo_en_uvr'].apply(lambda x: x == 0 or x == '0').sum()

print(f"El número de valores 0 en 'valor_avaluo_en_uvr' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")

El número de valores 0 en 'valor_avaluo_en_uvr' es: 0 que representa el 0.00% de los datos


In [50]:
# Esta si no tiene problemas y será limpiada por si acaso, similar a 'valor_total_avaluo'
# Se convierte a float la columna 'valor_avaluo_en_uvr'
convert_to_float(df, ['valor_avaluo_en_uvr'])

### Limpieza de area_valorada

In [51]:
convert_to_float(df, ['area_valorada'])

In [52]:
# Contar cuántos valores son 0 en la columna
conteo = df['area_valorada'].apply(lambda x: x == 0).sum()

print(f"El número de valores 0 en 'area_valorada' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")

El número de valores 0 en 'area_valorada' es: 44 que representa el 0.41% de los datos


In [53]:
# Puede que no solo sean areas de 0, tambien pueden ser valores mal ingresados
# Volvemos a calcular los percentiles
percentile_1 = df['area_valorada'].quantile(0.01)
percentile_99 = df['area_valorada'].quantile(0.99)

print(f"Usando los percentiles 1 y 99, los límites inferior y superior son: {percentile_1} y {percentile_99}")

Usando los percentiles 1 y 99, los límites inferior y superior son: 30.012 y 399.39999999999964


In [54]:
print(f"DataFrame original tenía {len(df)} filas")
df = df[df['area_valorada'] <= percentile_99]
df = df[df['area_valorada'] >= percentile_1]
print(f"Después de aplicar el filtro, el DataFrame tiene {len(df)} filas")

DataFrame original tenía 10861 filas
Después de aplicar el filtro, el DataFrame tiene 10643 filas


### Limpieza tipo_deposito

In [55]:
# Define una función para limpiar los valores de la columna 'tipo_deposito'
def limpiar_deposito(value):
    # Convierte el valor de entrada a una cadena para asegurar un procesamiento consistente
    value = str(value)
    
    # Si el valor indica un depósito exclusivo o privado, retorna 1
    if value == 'Exclusivo' or value == 'Privado':
        return 1
    
    # Si el valor indica la ausencia de un depósito, retorna 0
    elif value == 'No' or value == '0':
        return 0
    
    # Para cualquier otro valor, retorna None para indicar un valor indefinido o faltante
    else:
        return None
    
# Aplica la función de limpieza a cada valor en la columna 'tipo_deposito' del DataFrame 'df'
# Esto estandariza los valores de la columna a 1, 0 o None, indicando el tipo de depósito o su ausencia
df['tipo_deposito'] = df['tipo_deposito'].apply(limpiar_deposito)

### Limpieza de observaciones_indice_ocupacion, a pesar de que el campo es interesante para determinar cuantas habitaciones o posibles personas pueden vivir en el inmueble, tiene demasiados 0 y eso representa el 80% de la data, esta columna será eliminada

In [56]:
# Contar cuántos valores son 0 en la columna
conteo = df['observaciones_indice_ocupacion'].apply(lambda x: x == 0 or x == '0').sum()

print(f"El número de valores 0 en 'area_valorada' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")

El número de valores 0 en 'area_valorada' es: 8463 que representa el 79.52% de los datos


In [57]:
# Eliminar columna observaciones_indice_ocupacion
df = df.drop(columns=['observaciones_indice_ocupacion'])

### Limpieza de valor_area_construccion, pasa lo mismo, el 70% de la data son 0, se elimina la columna

In [58]:
# Contar cuántos valores son 0 en la columna
conteo = df['valor_area_construccion'].apply(lambda x: x == 0 or x == '0').sum()

print(f"El número de valores 0 en 'valor_area_construccion' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")

El número de valores 0 en 'valor_area_construccion' es: 7596 que representa el 71.37% de los datos


In [59]:
# Eliminar columna valor_area_construccion
df = df.drop(columns=['valor_area_construccion'])

### Limpieza de area_construccion, también es data incompleta, eliminar

In [60]:
# Contar cuántos valores son 0 en la columna
conteo = df['area_construccion'].apply(lambda x: x == 0 or x == '0').sum()

print(f"El número de valores 0 en 'area_construccion' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")

El número de valores 0 en 'area_construccion' es: 7594 que representa el 71.35% de los datos


In [61]:
# Eliminar columna valor_area_construccion
df = df.drop(columns=['area_construccion'])

### Con esto se termina el análisis de las columnas numéricas, el resto será forzado a ser numérico y sus valores null serán dropeados

In [62]:
### actualizar la lista de numéricos
columnas_eliminar_3 = ["area_construccion", "valor_area_construccion", "observaciones_indice_ocupacion", "valor_area_privada"]
numericos_filtrados = [col for col in numericos_filtrados if col not in columnas_eliminar_3]

In [63]:
for column in numericos_filtrados:
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [64]:
# Se revisa si la limpieza de estrato generó valores nulos:
null_counts = df.isnull().sum()
sorted_null_counts = null_counts.sort_values(ascending=False)
print(sorted_null_counts)

print(f"Antes el dataframe cuenta con {len(df)} filas")

df = df.dropna()

print(f"Después el dataframe cuenta con {len(df)} filas")

observaciones_indice_construccion    416
matricula_garaje_2                   403
valor_area_terreno                   362
area_terreno                         360
metodo_valuacion_9                   357
                                    ... 
area_libre                             0
valor_total_avaluo                     0
valor_avaluo_en_uvr                    0
Longitud                               0
Latitud                                0
Length: 198, dtype: int64
Antes el dataframe cuenta con 10643 filas
Después el dataframe cuenta con 8282 filas


In [65]:
#Una última cosa, se revisará en las columnas numericos_filtrados si la mayoria de sus valores son 0, si es así se eliminarán
zero_counts = {col: (df[col] == 0).sum() for col in numericos_filtrados}

# Convertir el diccionario a un DataFrame y ordenar por el número de ceros de mayor a menor
zero_counts_df = pd.DataFrame(list(zero_counts.items()), columns=['Columna', 'Conteo de Ceros'])
zero_counts_df = zero_counts_df.sort_values(by='Conteo de Ceros', ascending=False)

all_zeros_columns = [col for col, count in zero_counts.items() if count == len(df)]

In [66]:
# Estas columnas son 0 en toda la longitud del df, por lo que no aportan nada al análisis
all_zeros_columns

['matricula_garaje_2',
 'garaje_paralelo_2',
 'numero_garaje_3',
 'matricula_garaje_3',
 'garaje_cubierto_3',
 'garaje_doble_3',
 'garaje_paralelo_3',
 'garaje_servidumbre_3',
 'numero_garaje_4',
 'matricula_garaje_4',
 'garaje_cubierto_4',
 'garaje_doble_4',
 'garaje_paralelo_4',
 'garaje_servidumbre_4',
 'numero_garaje_5',
 'matricula_garaje_5',
 'garaje_cubierto_5',
 'garaje_doble_5',
 'garaje_paralelo_5',
 'garaje_servidumbre_5',
 'numero_deposito_2',
 'matricula_inmobiliaria_deposito_2',
 'numero_deposito_3',
 'matricula_inmobiliaria_deposito_3',
 'numero_deposito_4',
 'matricula_inmobiliaria_deposito_4',
 'numero_deposito_5',
 'matricula_inmobiliaria_deposito_5',
 'metodo_valuacion_1',
 'concepto_del_metodo_1',
 'metodo_valuacion_2',
 'concepto_del_metodo_2',
 'metodo_valuacion_4',
 'concepto_del_metodo_4',
 'metodo_valuacion_7',
 'concepto_del_metodo_7',
 'metodo_valuacion_9',
 'concepto_del_metodo_9']

In [67]:
df = df.drop(columns=all_zeros_columns)

### Igual siguen existiendo errores debido a que no se analizaron todas las columnas, si la cantidad de errore era inferior al 5% esas columnas se forzarán a convertirse en numéricas y los valores null eliminados

### Ahora se revisarán las categóricas, por cuestiones de tiempo solo se revisarán las columnas que contengan menos de 5 valores

In [68]:
len(categoricos)

100

In [69]:
categoricos

['objeto',
 'motivo',
 'proposito',
 'tipo_avaluo',
 'tipo_credito',
 'departamento_inmueble',
 'municipio_inmueble',
 'barrio',
 'sector',
 'direccion_inmueble_informe',
 'alcantarillado_en_el_sector',
 'acueducto_en_el_sector',
 'gas_en_el_sector',
 'energia_en_el_sector',
 'telefono_en_el_sector',
 'vias_pavimentadas',
 'sardineles_en_las_vias',
 'andenes_en_las_vias',
 'barrio_legal',
 'topografia_sector',
 'condiciones_salubridad',
 'transporte',
 'demanda_interes',
 'paradero',
 'alumbrado',
 'arborizacion',
 'alamedas',
 'ciclo_rutas',
 'nivel_equipamiento_comercial',
 'descripcion_general_sector',
 'perspectivas_de_valorizacion',
 'actualidad_edificadora',
 'comportamiento_oferta_demanda',
 'alcantarillado_en_el_predio',
 'acueducto_en_el_predio',
 'gas_en_el_predio',
 'energia_en_el_predio',
 'telefono_en_el_predio',
 'tipo_inmueble',
 'uso_actual',
 'clase_inmueble',
 'ocupante',
 'sometido_a_propiedad_horizontal',
 'altura_permitida',
 'aislamiento_posterior',
 'aislamiento_

In [70]:
# Lista para contener las columnas con más de 5 valores únicos
columnas_con_mas_de_5_unicos = []

# Iterar sobre la lista de columnas categóricas
for col in categoricos:
    # Verificar si la columna existe en el DataFrame 'df'
    if col in df.columns:
        # Calcular el número de valores únicos en la columna
        valores_unicos = df[col].nunique()
        # Si el número de valores únicos es mayor a 5, añadir el nombre de la columna a la lista
        if valores_unicos > 5:
            columnas_con_mas_de_5_unicos.append(col)

# Mostrar las columnas con más de 5 valores únicos
print("Columnas con más de 5 valores únicos:")
print(columnas_con_mas_de_5_unicos)

Columnas con más de 5 valores únicos:
['departamento_inmueble', 'municipio_inmueble', 'barrio', 'direccion_inmueble_informe', 'descripcion_general_sector', 'perspectivas_de_valorizacion', 'actualidad_edificadora', 'comportamiento_oferta_demanda', 'tipo_inmueble', 'uso_actual', 'clase_inmueble', 'ocupante', 'observaciones_generales_inmueble', 'area_actividad', 'uso_principal_ph', 'estructura', 'cubierta', 'fachada', 'material_de_construccion', 'detalle_material', 'observaciones_estructura', 'observaciones_dependencias', 'observaciones_generales_construccion', 'concepto_del_metodo_5']


In [71]:
categoricos_filtrados = [col for col in categoricos if col not in columnas_con_mas_de_5_unicos]
df = df.drop(columns=columnas_con_mas_de_5_unicos)

In [72]:
def categorize_numeric_percentage(categoricos, df):
    # Inicializa un diccionario para almacenar los nombres de las columnas y sus porcentajes de valores numéricos
    column_percentages = {}

    # Calcula el porcentaje de valores numéricos para cada columna categórica
    for column in categoricos:
        if column in df.columns:
            # Intenta convertir la columna a numéricos, tratando los no convertibles como NaN
            numeric_values = pd.to_numeric(df[column], errors='coerce')
            # Cuenta el número de valores numéricos en la columna
            num_numeric = numeric_values.notna().sum()
            # Obtiene el número total de filas en el DataFrame
            total_df = len(df)
            # Calcula el porcentaje de valores numéricos en la columna
            percentage = (num_numeric / total_df) * 100
            # Almacena el porcentaje calculado en el diccionario
            column_percentages[column] = percentage

    # Convierte el diccionario a un DataFrame para una mejor visualización
    percentages_df = pd.DataFrame(list(column_percentages.items()), columns=['Column', 'Percentage'])

    # Ordena el DataFrame por porcentaje de forma descendente
    sorted_percentages_df = percentages_df.sort_values(by='Percentage', ascending=False)

    # Devuelve el DataFrame ordenado
    return sorted_percentages_df

In [73]:
percentages = categorize_numeric_percentage(categoricos_filtrados, df)
percentages.head(25)

,Column,Percentage
69,metodo_valuacion_5,27.988409
41,garaje_visitantes,25.114707
39,tanque_de_agua,25.114707
37,bicicletero,25.114707
36,citofono,25.114707
38,piscina,25.114707
40,club_house,25.114707
35,porteria,25.114707
33,condicion_ph,24.824922
45,administracion,23.641632


### Similar al caso anterior aquí tenemos una variables categoricas pero que han insertado valores numéricos erróneos, por lo que dichas columnas se limpiarán siempre y cuando sea superior al 5%

### Limpieza metodo_valuacion_5, este realmente no nos indica mucho, solamente la Comparación de Mercado PH en Venta, no es información relevante, se elimina

In [74]:
df = df.drop(columns=['metodo_valuacion_5'])
categoricos_filtrados.remove("metodo_valuacion_5")

### La limpieza de garaje_visitantes, realmente debería ser una numérica, donde si es 1 y no es 0, solo que hay mas veces las palabras por eso fué detectada como categórica. Lo mismo pasa para tanque_de_agua, bicicletero, citofono, piscina, club_house, porteria, administracion, area_libre, teatrino, rph, sauna

In [75]:
# Aplicar la función 'limpiar_garaje' a varias columnas del DataFrame 'df' para estandarizar sus valores
df['garaje_visitantes'] = df['garaje_visitantes'].apply(limpiar_garaje)
df['tanque_de_agua'] = df['tanque_de_agua'].apply(limpiar_garaje)
df['bicicletero'] = df['bicicletero'].apply(limpiar_garaje)
df['citofono'] = df['citofono'].apply(limpiar_garaje)
df['piscina'] = df['piscina'].apply(limpiar_garaje)
df['club_house'] = df['club_house'].apply(limpiar_garaje)
df['porteria'] = df['porteria'].apply(limpiar_garaje)
df['administracion'] = df['administracion'].apply(limpiar_garaje)
df['area_libre'] = df['area_libre'].apply(limpiar_garaje)
df['teatrino'] = df['teatrino'].apply(limpiar_garaje)
df['rph'] = df['rph'].apply(limpiar_garaje)
df['sauna'] = df['sauna'].apply(limpiar_garaje)

# Lista de columnas a eliminar de la lista de columnas categóricas
eliminar_de_categoricos = ["garaje_visitantes", "tanque_de_agua", "bicicletero", "citofono", 
                           "piscina", "club_house", "porteria", "administracion", "area_libre", "teatrino", "rph", "sauna"]

# Filtrar las columnas categóricas para excluir las especificadas en 'eliminar_de_categoricos'
categoricos_filtrados = [col for col in categoricos_filtrados if col not in eliminar_de_categoricos]

### Limpieza de condicion_ph, esto tiene valores Terreno, Construcción, no se si el 0 haga referencia a Terreno, haciendo una revisión manual no me queda claro por qué los 0, si solo tieen dos categorias y adicionalmente hay descripción física de la vivienda con sus espacios, por lo que al no tener claro las cosas elimino dicha columna

In [76]:
df = df.drop(columns=['condicion_ph'])
categoricos_filtrados.remove("condicion_ph")

### Con esto se limpiarían todos los categóricos encontrados

In [77]:
percentages = categorize_numeric_percentage(categoricos_filtrados, df)
percentages.head(25)

,Column,Percentage
24,telefono_en_el_predio,2.523545
22,gas_en_el_predio,1.412702
32,predio_subdividido_fisicamente,0.543347
30,indice_ocupacion,0.543347
31,indice_construccion,0.543347
29,antejardin,0.543347
28,aislamiento_lateral,0.543347
27,aislamiento_posterior,0.543347
26,altura_permitida,0.543347
20,alcantarillado_en_el_predio,0.398454


### Ahora se revisan las categoricas de 2 valores, muy probablemente esto sea un Si y No, cosa que no nos sirve para modelar, es mejor cambiarlo a numérica como 1 y 0

In [78]:
# List to hold columns with more than 5 unique values
columns_with_2_unique = []

# Iterate over the list of categorical columns
for col in categoricos_filtrados:
    if col in df.columns:
        unique_values = df[col].nunique()
        if unique_values == 2:
            columns_with_2_unique.append(col)

# Display the columns with more than 5 unique values
print("Columns with 2 unique values:")
print(columns_with_2_unique)

Columns with 2 unique values:
['sector', 'alcantarillado_en_el_sector', 'acueducto_en_el_sector', 'gas_en_el_sector', 'energia_en_el_sector', 'telefono_en_el_sector', 'vias_pavimentadas', 'sardineles_en_las_vias', 'andenes_en_las_vias', 'barrio_legal', 'condiciones_salubridad', 'paradero', 'alumbrado', 'arborizacion', 'alamedas', 'ciclo_rutas', 'sometido_a_propiedad_horizontal', 'vigilancia_privada']


### De estas Columnas solo condiciones_salubridad y sector tiene otros valores, por lo que serán limpiados cambiando en sector urbano=1 y rural=0, y en condiciones_salubridad como buenas=1, regulares=0

In [79]:
# Elimina 'sector' de la lista 'columns_with_2_unique'
columns_with_2_unique.remove("sector")
# Elimina 'condiciones_salubridad' de la lista 'columns_with_2_unique'
columns_with_2_unique.remove("condiciones_salubridad")

### Limpiar sector

In [80]:
# Define una función para limpiar y transformar los valores de la columna 'sector'
def limpiar_sector(value):
    # Si el valor es 'Urbano', retorna 1
    if value == 'Urbano':
        return 1
    # Si el valor es 'Rural', retorna 0
    elif value == 'Rural':
        return 0
    
# Aplica la función 'limpiar_sector' a cada valor en la columna 'sector' del DataFrame 'df'
# Esto convierte los valores textuales 'Urbano' y 'Rural' a valores numéricos 1 y 0, respectivamente
df['sector'] = df['sector'].apply(limpiar_sector)
categoricos_filtrados.remove("sector")

### Limpiar condiciones_salubridad

In [81]:
# Define una función para limpiar y transformar los valores de la columna 'condiciones_salubridad'
def limpiar_condiciones_salubridad(value):
    # Si el valor es 'Buenas', retorna 1
    if value == 'Buenas':
        return 1
    # Si el valor es 'Regulares', retorna 0
    elif value == 'Regulares':
        return 0
    # Para cualquier otro valor, retorna None para indicar un valor indefinido o faltante
    else:
        return None
    
# Aplica la función 'limpiar_condiciones_salubridad' a cada valor en la columna 'condiciones_salubridad' del DataFrame 'df'
# Esto convierte los valores textuales 'Buenas' y 'Regulares' a valores numéricos 1 y 0, respectivamente, y maneja valores indefinidos
df['condiciones_salubridad'] = df['condiciones_salubridad'].apply(limpiar_condiciones_salubridad)
categoricos_filtrados.remove("condiciones_salubridad")

### Limpieza del resto

In [82]:
# Itera sobre cada columna en la lista 'columns_with_2_unique'
for column in columns_with_2_unique:
    # Aplica la función 'limpiar_garaje' a cada valor en la columna actual del DataFrame 'df'
    # Esto estandariza los valores de las columnas seleccionadas, posiblemente convirtiendo textos a valores numéricos o binarios
    df[column] = df[column].apply(limpiar_garaje)

# Actualiza la lista 'categoricos_filtrados' excluyendo las columnas procesadas en 'columns_with_2_unique'
# Esto se hace para mantener solo las columnas categóricas que no han sido transformadas o no necesitan limpieza
categoricos_filtrados = [col for col in categoricos_filtrados if col not in columns_with_2_unique]

### Ahora se revisa cuantos valores únicos por cada columna categorica hay:

In [83]:
# Dictionary to hold columns with 5 or fewer unique values
filtered_columns = {}

# Iterate over the list of categorical columns
for col in categoricos_filtrados:
    if col in df.columns:
        unique_values = df[col].nunique()
        if unique_values <= 5:
            filtered_columns[col] = unique_values

# Display the columns with 5 or fewer unique values
print("Columns with 5 or fewer unique values:")
for col, unique_count in filtered_columns.items():
    print(f"{col}: {unique_count} unique values")

Columns with 5 or fewer unique values:
topografia_sector: 4 unique values
transporte: 3 unique values
demanda_interes: 5 unique values
nivel_equipamiento_comercial: 4 unique values
alcantarillado_en_el_predio: 3 unique values
acueducto_en_el_predio: 3 unique values
gas_en_el_predio: 3 unique values
energia_en_el_predio: 3 unique values
telefono_en_el_predio: 3 unique values
altura_permitida: 4 unique values
aislamiento_posterior: 3 unique values
aislamiento_lateral: 3 unique values
antejardin: 3 unique values
indice_ocupacion: 4 unique values
indice_construccion: 4 unique values
predio_subdividido_fisicamente: 3 unique values
ajustes_sismoresistentes: 3 unique values
tipo_fachada: 3 unique values
estructura_reforzada: 4 unique values
danos_previos: 5 unique values
iluminacion: 3 unique values
ventilacion: 3 unique values
irregularidad_planta: 3 unique values
irregularidad_altura: 3 unique values
estado_acabados_pisos: 4 unique values
calidad_acabados_pisos: 4 unique values
estado_acaba

### Revisando danos_previos, a pesar de tener data relativamente decente hay una gran cantidad de data que es No disponible, por lo que sacaré esta variable del análisis, lo mismo sucede para estructura_reforzada, ajustes_sismoresistentes, irregularidad_altura, irregularidad_planta

In [84]:

conteo = df['danos_previos'].apply(lambda x: x == 'No disponible').sum()
print(f"El número de valores No disponible en 'danos_previos' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")



conteo = df['estructura_reforzada'].apply(lambda x: x == 'No disponible').sum()
print(f"El número de valores No disponible en 'estructura_reforzada' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")


conteo = df['ajustes_sismoresistentes'].apply(lambda x: x == 'No disponible').sum()
print(f"El número de valores No disponible en 'ajustes_sismoresistentes' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")



conteo = df['irregularidad_altura'].apply(lambda x: x == 'No disponible').sum()
print(f"El número de valores No disponible en 'irregularidad_altura' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")


conteo = df['irregularidad_planta'].apply(lambda x: x == 'No disponible').sum()
print(f"El número de valores No disponible en 'irregularidad_planta' es: {conteo} que representa el {conteo/len(df)*100:.2f}% de los datos")



El número de valores No disponible en 'danos_previos' es: 5480 que representa el 66.17% de los datos
El número de valores No disponible en 'estructura_reforzada' es: 5989 que representa el 72.31% de los datos
El número de valores No disponible en 'ajustes_sismoresistentes' es: 7422 que representa el 89.62% de los datos
El número de valores No disponible en 'irregularidad_altura' es: 4311 que representa el 52.05% de los datos
El número de valores No disponible en 'irregularidad_planta' es: 4322 que representa el 52.19% de los datos


In [85]:
# Esto quierre decir que de estas 3 columnas no se completó la información debidamente y aunque podría ser importante para el precio
# entiendo la dificultad en encontrar dicha información
df = df.drop(columns=['danos_previos'])
categoricos_filtrados.remove("danos_previos")

df = df.drop(columns=['estructura_reforzada'])
categoricos_filtrados.remove("estructura_reforzada")

df = df.drop(columns=['ajustes_sismoresistentes'])
categoricos_filtrados.remove("ajustes_sismoresistentes")

df = df.drop(columns=['irregularidad_altura'])
categoricos_filtrados.remove("irregularidad_altura")

### Se identificó que predio_subdividido_fisicamente, telefono_en_el_predio, energia_en_el_predio, gas_en_el_predio, acueducto_en_el_predio, alcantarillado_en_el_predio,  puede ser procesado por la función garaje, 

In [86]:
# Aplica la función 'limpiar_garaje' a varias columnas del DataFrame 'df' para estandarizar sus valores
# Esta función convierte valores textuales como 'Si' o 'No' a valores numéricos 1 y 0, respectivamente
df['predio_subdividido_fisicamente'] = df['predio_subdividido_fisicamente'].apply(limpiar_garaje)
df['telefono_en_el_predio'] = df['telefono_en_el_predio'].apply(limpiar_garaje)
df['energia_en_el_predio'] = df['energia_en_el_predio'].apply(limpiar_garaje)
df['gas_en_el_predio'] = df['gas_en_el_predio'].apply(limpiar_garaje)
df['acueducto_en_el_predio'] = df['acueducto_en_el_predio'].apply(limpiar_garaje)
df['alcantarillado_en_el_predio'] = df['alcantarillado_en_el_predio'].apply(limpiar_garaje)

# Elimina las columnas procesadas de la lista 'categoricos_filtrados'
# Esto se hace porque las columnas ya han sido convertidas a valores numéricos y ya no se consideran categóricas
categoricos_filtrados.remove("predio_subdividido_fisicamente")
categoricos_filtrados.remove("telefono_en_el_predio")
categoricos_filtrados.remove("energia_en_el_predio")
categoricos_filtrados.remove("gas_en_el_predio")
categoricos_filtrados.remove("acueducto_en_el_predio")
categoricos_filtrados.remove("alcantarillado_en_el_predio")

### Se identificó que indice_construccion, indice_ocupacion, altura_permitida podría ser reducido de 4 a 3 categorías, una es 0, por lo que esa pasa a No Aplica

In [87]:
# Define una función para limpiar y transformar los valores de la columna 'indice_construccion'
def limpiar_indice_construccion(value):
    # Si el valor contiene "0", retorna "No Aplica" indicando que no aplica el índice de construcción
    if "0" in value:
        return "No Aplica"
    # Si el valor contiene "1", retorna "Aplica" indicando que aplica el índice de construcción
    elif "1" in value:
        return "Aplica"
    # Si el valor no contiene ni "0" ni "1", retorna el valor original sin cambios
    else:
        return value
    
# Aplica la función 'limpiar_indice_construccion' a cada valor en la columna 'indice_construccion' del DataFrame 'df'
# Esto transforma los valores numéricos en valores descriptivos más comprensibles ("No Aplica" o "Aplica")
df['indice_construccion'] = df['indice_construccion'].apply(limpiar_indice_construccion)
df['indice_ocupacion'] = df['indice_ocupacion'].apply(limpiar_indice_construccion)
df['altura_permitida'] = df['altura_permitida'].apply(limpiar_indice_construccion)

### antejardin, aislamiento_lateral, aislamiento_posterior tiene una categoria Aplica, No Aplica, esto puede ser llevado a numérico como 1 y 0

In [88]:
# Define una función para limpiar y transformar los valores de las columnas relacionadas con aplicabilidad
def limpiar_aplicas(value):
    # Si el valor contiene "Aplica", retorna 1 indicando que la característica aplica
    if "Aplica" in value:
        return 1
    # Si el valor contiene "No Aplica", retorna 0 indicando que la característica no aplica
    elif "No Aplica" in value:
        return 0
    # Para cualquier otro valor, retorna None para indicar un valor indefinido o faltante
    elif "0" in value:
        return 0
    else:
        return None
    
# Aplica la función 'limpiar_aplicas' a las columnas 'antejardin', 'aislamiento_lateral', y 'aislamiento_posterior' del DataFrame 'df'
# Esto convierte los valores textuales "Aplica" y "No Aplica" a valores numéricos 1 y 0, respectivamente, y maneja valores indefinidos
df['antejardin'] = df['antejardin'].apply(limpiar_aplicas)
df['aislamiento_lateral'] = df['aislamiento_lateral'].apply(limpiar_aplicas)
df['aislamiento_posterior'] = df['aislamiento_posterior'].apply(limpiar_aplicas)

categoricos_filtrados.remove("antejardin")
categoricos_filtrados.remove("aislamiento_lateral")
categoricos_filtrados.remove("aislamiento_posterior")

### demanda_interes requiere mínima limpieza demanda_interes

In [89]:
# Define una función para corregir errores de codificación en la columna 'demanda_interes'
def limpiar_demanda_interes(value):
    # Si el valor contiene "DÃ©bil" (resultado de un error de codificación), corrige el texto a "Débil"
    if "DÃ©bil" in value:
        return "Débil"
    # Si no hay error de codificación en el valor, lo retorna sin cambios
    else:
        return value
    
# Aplica la función 'limpiar_demanda_interes' a cada valor en la columna 'demanda_interes' del DataFrame 'df'
# Esto corrige los errores de codificación específicamente para los valores que deberían ser "Débil"
df['demanda_interes'] = df['demanda_interes'].apply(limpiar_demanda_interes)

### Se vuelven a revisar los porcentajes de categóricos con valores numéricos

In [90]:
percentages = categorize_numeric_percentage(categoricos_filtrados, df)
percentages.head(25)

,Column,Percentage
0,topografia_sector,0.0
1,transporte,0.0
2,demanda_interes,0.0
3,nivel_equipamiento_comercial,0.0
4,altura_permitida,0.0
5,indice_ocupacion,0.0
6,indice_construccion,0.0
7,tipo_fachada,0.0
8,iluminacion,0.0
9,ventilacion,0.0


### Con esto estaría casi completa la limpieza de los datos, finalmente esa lista de variables categóricas se expandirán en columnas adicionales para finalmente tener toda la información de manera numérica

In [91]:
# Se hace una última limpieza a categoricos_filtrados
categoricos_filtrados = [col for col in categoricos_filtrados if col not in columnas_eliminar_2]

In [92]:
df_expanded = pd.get_dummies(df, columns=categoricos_filtrados, drop_first=False)
df_expanded = df_expanded.astype(float)

In [93]:
df_expanded

,id,sector,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,telefono_en_el_sector,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,...,estado_acabados_cocina_Regular,estado_acabados_cocina_Sin acabados,calidad_acabados_cocina_Integral,calidad_acabados_cocina_Semi-Integral,calidad_acabados_cocina_Sencillo,calidad_acabados_cocina_Sin Acabados,tipo_garaje_Comunal,tipo_garaje_Exclusivo,tipo_garaje_No Tiene,tipo_garaje_Privado
1,10570.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,14600.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5967.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,15814.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6,8190.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11564,8160.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11565,16114.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
11567,7419.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
11568,7687.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Esto crece de nuevo la cantidad de columnas del df, pero los valores como por ejemplo acabados: buenos, malos, regulares, se convierten en 3 columnas llamada acabados_buenos, acabados_malos, con valores 0 y 1

### Hace falta una última implementación, el caso de los lugares de interés, para esto se usará el punto de cada vivienda y se trazará un área cuyo centro sea el punto, si algún punto de interés se encuentra dentro del área se añadira a una nueva columna de lugares de interés.

In [94]:
# from geopy.distance import geodesic
# puntos_interes_df =  pd.read_csv("data_original/precios_vivienda/PuntosInteres (2).csv", encoding='utf-8')

# df = df.dropna(subset=['Longitud', 'Latitud'])
# df = df[(df['Latitud'].between(-90, 90)) & (df['Longitud'].between(-180, 180))]

# puntos_interes_df = puntos_interes_df.dropna(subset=['Longitud', 'Latitud'])
# puntos_interes_df = puntos_interes_df[(puntos_interes_df['Latitud'].between(-90, 90)) & (puntos_interes_df['Longitud'].between(-180, 180))]

# # Definir una función para verificar si un punto está dentro de un radio de 1 km
# def is_within_radius(lon1, lat1, lon2, lat2, radius_km=1):
#     return geodesic((lat1, lon1), (lat2, lon2)).km <= radius_km

In [95]:
# # # Definir una función para verificar si un punto está dentro de un radio de 1 km
# # def is_within_radius(lon1, lat1, lon2, lat2, radius_km=1):
# #     return geodesic((lat1, lon1), (lat2, lon2)).km <= radius_km

# def count_cercanos_for_row(index, row, puntos_df):
#     count_cercano = 0
#     longitud_df = row['Longitud']
#     latitud_df = row['Latitud']
    
#     for _, punto in puntos_df.iterrows():
#         longitud_punto = punto['Longitud']
#         latitud_punto = punto['Latitud']
#         if is_within_radius(longitud_df, latitud_df, longitud_punto, latitud_punto):
#             count_cercano += 1
    
#     return index, count_cercano

# def parallel_process(df, puntos_df):
#     results = []
    
#     def worker(index, row):
#         return count_cercanos_for_row(index, row, puntos_df)
    
#     with ThreadPoolExecutor() as executor:
#         futures = [executor.submit(worker, index, row) for index, row in df.iterrows()]
        
#         for future in futures:
#             index, count_cercano = future.result()
#             results.append((index, count_cercano))
    
#     for index, count_cercano in results:
#         df.at[index, 'ConteoPuntosInteresCercanos'] = count_cercano

# # Add the new column to the DataFrame
# df['ConteoPuntosInteresCercanos'] = 0

# # Call the function to process the DataFrame in parallel
# parallel_process(df, puntos_interes_df)

### Lastimosamente por cuestiones de tiempo aparentemente no podré implementar esta solución, esto se debe a que hay que compara los 8000 valores del df con los 40000 valores que hay en el df de latitudes, esto podría demorar al menos un par de horas o días y no cuento con el tiempo

## 3. Preparación para el modelo <a id="third"></a>

In [96]:
# Paso 1: Extraer aleatoriamente el 20% del DataFrame para un conjunto de prueba separado
df_test_only, df_remaining = train_test_split(df_expanded, test_size=0.8, random_state=1)

In [97]:
# Paso 2: Usar el 80% restante para dividirlo en conjuntos de entrenamiento y validación
X = df_remaining.drop(columns=['valor_total_avaluo'])  # Datos de entrada sin la columna objetivo
y = df_remaining['valor_total_avaluo']  # Columna objetivo

In [98]:
# Dividir el 80% restante en conjuntos de entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [99]:
print(f"Tamaño del conjunto de prueba separado: {len(df_test_only)}")
print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de validación: {len(X_test)}")

Tamaño del conjunto de prueba separado: 1656
Tamaño del conjunto de entrenamiento: 5300
Tamaño del conjunto de validación: 1326


In [100]:

# Definición de los regresores
regressors = {
    'LR': LinearRegression(),
    'KNN': KNeighborsRegressor(),
    'RFC': RandomForestRegressor(),
    'ETR': ExtraTreesRegressor()
}

# Definición de los hiperparámetros para búsqueda en cuadrícula
param_grids = {
    'LR': {
        'fit_intercept': [True, False]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 10, 15],
        'weights': ['uniform', 'distance']
    },
    'RFC': {
        'n_estimators': [50, 100, 150, 200, 300],
        'max_depth': [None, 10, 20, 30, 40]
    },
    'ETR': {
        'n_estimators': [50, 100, 150, 200, 300],
        'max_features': [None, 'sqrt', 'log2']
    }
}

best_params = {}
best_models = {}

# Función para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    nonzero_elements = y_true != 0
    mape = np.mean(np.abs((y_true[nonzero_elements] - y_pred[nonzero_elements]) / y_true[nonzero_elements])) * 100
    return mape

# Iterar sobre los regresores
for name, reg in regressors.items():
    # Crear GridSearchCV para cada regresor
    grid_search = GridSearchCV(reg, param_grids[name], cv=5, scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)
    
    # Realizar búsqueda de hiperparámetros en los datos de entrenamiento
    grid_search.fit(X_train, y_train)
    
    # Obtener los mejores parámetros y el mejor modelo
    best_params[name] = grid_search.best_params_
    best_models[name] = grid_search.best_estimator_
    
    # Realizar predicciones en el conjunto de prueba con el mejor modelo
    y_pred = grid_search.predict(X_test)
    
    # Calcular el MAPE
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    # Imprimir el MAPE
    print(f"MAPE para {name}: {mape:.2f}%")


Fitting 5 folds for each of 2 candidates, totalling 10 fits
MAPE para LR: 0.65%
Fitting 5 folds for each of 10 candidates, totalling 50 fits
MAPE para KNN: 12.48%
Fitting 5 folds for each of 25 candidates, totalling 125 fits
MAPE para RFC: 0.30%
Fitting 5 folds for each of 15 candidates, totalling 75 fits
MAPE para ETR: 0.28%


## 4. Resultados <a id="fourth"></a>

In [101]:
# Imprimir los mejores parámetros para cada regresor
print("Mejores parámetros para cada regresor:")
for name, params in best_params.items():
    print(f"{name}: {params}")

Mejores parámetros para cada regresor:
LR: {'fit_intercept': True}
KNN: {'n_neighbors': 5, 'weights': 'distance'}
RFC: {'max_depth': 40, 'n_estimators': 300}
ETR: {'max_features': None, 'n_estimators': 300}


In [102]:
# Uso de los datos de prueba separados para evaluar el mejor modelo
X_test_only = df_test_only.drop(columns=['valor_total_avaluo'])  # Elimina la columna objetivo
y_test_only = df_test_only['valor_total_avaluo']  # Extrae la columna objetivo

# Realizar predicciones usando el mejor modelo
# Seleccionar el modelo con el mejor MAPE de los modelos ajustados
best_model_name = min(best_models, key=lambda k: mean_absolute_percentage_error(y_test, best_models[k].predict(X_test)))
print(f"El mejor modelo es: {best_model_name}")
best_model = best_models[best_model_name]
y_pred_only = best_model.predict(X_test_only)

# Calcula el MAPE
mape = mean_absolute_percentage_error(y_test_only, y_pred_only)

# Calcula las métricas
# Para precisión y f1_score, necesitamos valores discretos, por lo que convertimos la predicción continua en binaria
umbral = 0.5
y_pred_binario = np.where(y_pred_only > umbral, 1, 0)
y_test_binario = np.where(y_test_only > umbral, 1, 0)

precision = precision_score(y_test_binario, y_pred_binario)
f1 = f1_score(y_test_binario, y_pred_binario)


# Imprime las métricas
print(f"MAPE: {mape:.2f}%")
print(f"Precisión: {precision:.4f}")
print(f"Puntaje F1: {f1:.4f}")


El mejor modelo es: ETR
MAPE: 0.27%
Precisión: 1.0000
Puntaje F1: 1.0000


In [103]:
# Crear DataFrame con las predicciones y valores reales
df_predictions = pd.DataFrame({
    'id': df_test_only['id'],  # Asegúrate de que 'id' está en df_test_only
    'valor_total_avaluo_predict': y_pred_only,
    'valor_total_avaluo': y_test_only
})

# Guardar el DataFrame en un archivo CSV
df_predictions.to_csv('predict_prices.csv', index=False)